In [1]:
pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install mediapipe==0.10.9

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pyqt5


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pyttsx3

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys
import cv2
import numpy as np
import csv
import datetime
import os
import pyttsx3
import time
import threading
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel, QLineEdit, QPushButton,
                             QVBoxLayout, QHBoxLayout, QMessageBox, QComboBox, QTextEdit, QInputDialog)
from PyQt5.QtCore import Qt, QTimer
from PyQt5.QtGui import QPixmap, QImage
import mediapipe as mp

# Initialize pose detection and drawing
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Initialize voice engine
engine = pyttsx3.init()
engine.setProperty('rate', 180)

# Final working version
import pyttsx3

engine = pyttsx3.init()
voices = engine.getProperty('voices')

# Try to select a female voice (usually index 1, but check your system)
for voice in voices:
    if "female" in voice.name.lower() or "zira" in voice.name.lower():
        engine.setProperty('voice', voice.id)
        break

engine.setProperty('rate', 150)

def speak(text):
    engine.say(text)
    engine.runAndWait()



def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return 360 - angle if angle > 180.0 else angle

def recommend_goal(bmi):
    if bmi < 18.5:
        return 'muscle gain'
    elif bmi < 25:
        return 'maintenance'
    else:
        return 'weight loss'

def recommend_exercises(goal):
    return {
        'squats': 5 if goal in ['weight loss', 'muscle gain'] else 3,
        'pushups': 5 if goal in ['weight loss', 'muscle gain'] else 3,
        'pullups': 3 if goal in ['weight loss', 'muscle gain'] else 2,
        'curls': 5 if goal in ['weight loss', 'muscle gain'] else 3
    }

def write_to_csv(name, age, gender, goal, squats, pushups, pullups, curls):
    file_exists = os.path.exists('FITNESS.csv')
    with open('FITNESS.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(['Name', 'Age', 'Gender', 'Goal', 'Squats', 'Pushups', 'Pullups', 'Curls', 'Timestamp'])
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        writer.writerow([name, age, gender, goal, squats, pushups, pullups, curls, timestamp])

def fetch_user_data(name):
    if not os.path.exists("FITNESS.csv"):
        return "No records found."
    with open("FITNESS.csv", 'r') as f:
        rows = list(csv.reader(f))
        header = rows[0]
        entries = [row for row in rows[1:] if row[0].lower() == name.lower()]
    return "\n".join([" | ".join(entry) for entry in entries[-3:]]) if entries else "No previous records found."

class FitnessApp(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('FitSavvy - Smarter fitness, better you')
        self.setGeometry(300, 300, 600, 600)

        self.name_input = QLineEdit()
        self.age_input = QLineEdit()
        self.gender_input = QComboBox()
        self.gender_input.addItems(['Male', 'Female'])
        self.height_input = QLineEdit()
        self.weight_input = QLineEdit()
        self.info_label = QLabel('Enter your details:')
        self.result_label = QLabel('')
        self.history_area = QTextEdit()
        self.history_area.setReadOnly(True)

        self.start_button = QPushButton('Start')
        self.start_button.clicked.connect(self.calculate_bmi)

        layout = QVBoxLayout()
        layout.addWidget(self.info_label)
        layout.addWidget(QLabel('Name:'))
        layout.addWidget(self.name_input)
        layout.addWidget(QLabel('Age:'))
        layout.addWidget(self.age_input)
        layout.addWidget(QLabel('Gender:'))
        layout.addWidget(self.gender_input)
        layout.addWidget(QLabel('Height (cm):'))
        layout.addWidget(self.height_input)
        layout.addWidget(QLabel('Weight (kg):'))
        layout.addWidget(self.weight_input)
        layout.addWidget(self.start_button)
        layout.addWidget(self.result_label)
        layout.addWidget(QLabel("Recent Workout History:"))
        layout.addWidget(self.history_area)
        self.setLayout(layout)

    def calculate_bmi(self):
        try:
            name = self.name_input.text()
            age = self.age_input.text()
            gender = self.gender_input.currentText()
            height = float(self.height_input.text()) / 100
            weight = float(self.weight_input.text())
            bmi = weight / (height ** 2)
            goal = recommend_goal(bmi)
            self.result_label.setText(f"Your BMI is {bmi:.2f}. Suggested goal: {goal}.")
            speak(f"Your BMI is {int(bmi)}. Suggested goal is {goal}. Do you want to start with this goal?")
            time.sleep(0.5)  # Optional
            self.ask_goal_confirmation(name, age, gender, goal)
            self.history_area.setText(fetch_user_data(name))
        except Exception as e:
            QMessageBox.warning(self, "Input Error", f"Invalid input: {str(e)}")


    def ask_goal_confirmation(self, name, age, gender, goal):
        reply = QMessageBox.question(self, 'Confirm Goal',
                                     f'Do you want to proceed with the suggested goal: {goal}?',
                                     QMessageBox.Yes | QMessageBox.No, QMessageBox.Yes)
        if reply == QMessageBox.Yes:
            self.close()
            start_workout(name, age, gender, goal)
        else:
            custom_goal, ok = QInputDialog.getItem(self, "Select Goal", "Choose your goal:",
                                                   ["weight loss", "muscle gain", "maintenance"], 0, False)
            if ok and custom_goal:
                self.close()
                start_workout(name, age, gender, custom_goal)

def start_workout(name, age, gender, goal):
    exercise_targets = recommend_exercises(goal)
    cap = cv2.VideoCapture(0)
    address="https://192.168.165.82:8080/video"
    cap.open(address)
    counters = {'squats': 0, 'pushups': 0, 'pullups': 0, 'curls': 0}
    stages = {'squats': None, 'pushups': None, 'pullups': None, 'curls': None}
    current = 'squats'
    speak("Starting workout with squats")

    with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.9) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                continue
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.pose_landmarks:
                lm = results.pose_landmarks.landmark
                try:
                    if current == 'squats':
                        angle = calculate_angle(
                            [lm[mp_pose.PoseLandmark.LEFT_HIP.value].x, lm[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                            [lm[mp_pose.PoseLandmark.LEFT_KNEE.value].x, lm[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
                            [lm[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, lm[mp_pose.PoseLandmark.LEFT_ANKLE.value].y])
                        if angle > 160:
                            stages[current] = 'up'
                        elif angle < 90 and stages[current] == 'up':
                            stages[current] = 'down'
                            counters[current] += 1
                            speak(f"Squat {counters[current]}")
                            if counters[current] >= exercise_targets[current]:
                                current = 'pushups'
                                speak("Next: Pushups")

                    elif current == 'pushups':
                        angle = calculate_angle(
                            [lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                            [lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
                            [lm[mp_pose.PoseLandmark.LEFT_WRIST.value].x, lm[mp_pose.PoseLandmark.LEFT_WRIST.value].y])
                        if angle > 160:
                            stages[current] = 'up'
                        elif angle < 90 and stages[current] == 'up':
                            stages[current] = 'down'
                            counters[current] += 1
                            speak(f"Pushup {counters[current]}")
                            if counters[current] >= exercise_targets[current]:
                                current = 'pullups'
                                speak("Next: Pullups")

                    elif current == 'pullups':
                        angle = calculate_angle(
                            [lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                            [lm[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, lm[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
                            [lm[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, lm[mp_pose.PoseLandmark.RIGHT_WRIST.value].y])
                        if angle > 160:
                            stages[current] = 'up'
                        elif angle < 90 and stages[current] == 'up':
                            stages[current] = 'down'
                            counters[current] += 1
                            speak(f"Pullup {counters[current]}")
                            if counters[current] >= exercise_targets[current]:
                                current = 'curls'
                                speak("Next: Curls")

                    elif current == 'curls':
                        angle = calculate_angle(
                            [lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                            [lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
                            [lm[mp_pose.PoseLandmark.LEFT_WRIST.value].x, lm[mp_pose.PoseLandmark.LEFT_WRIST.value].y])
                        if angle > 160:
                            stages[current] = 'down'
                        elif angle < 40 and stages[current] == 'down':
                            stages[current] = 'up'
                            counters[current] += 1
                            speak(f"Curl {counters[current]}")
                            if counters[current] >= exercise_targets[current]:
                                speak("Workout complete")
                                write_to_csv(name, age, gender, goal,
                                             counters['squats'], counters['pushups'],
                                             counters['pullups'], counters['curls'])
                                break
                except:
                    pass

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            for i, (ex, count) in enumerate(counters.items()):
                cv2.putText(image, f"{ex.capitalize()}: {count}/{exercise_targets[ex]}", (10, 30+i*30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

            cv2.imshow('Workout Tracker', image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    win = FitnessApp()
    win.show()
    sys.exit(app.exec_())


SystemExit: 0

/Users/yashrai/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
